In [51]:
def extract_ids_names(file):
    f = open(file, 'r')
    json_string = f.read()
    parsed_json = json.loads(json_string)
    inner_json_string = parsed_json['applist']['apps']
    ids = []
    names = []
    if file == 'v2.json':
        for dic in inner_json_string:
            ids.append(dic['appid'])
            names.append(dic['name'])
    else:
        for dic in inner_json_string['app']:
            ids.append(dic['appid'])
            names.append(dic['name'])
    return ids, names


In [75]:
import pandas as pd
import numpy as np
ids, names = extract_ids_names('v1.json')
ids2, names2 = extract_ids_names('v2.json')
ids.extend(ids2)
names.extend(names2)
df = pd.DataFrame({'ID': ids, 'Title': names }, columns=['ID', 'Title'], index =  np.arange(1, len(ids)+1))
df.head()
df.to_csv('all_steam_games.csv', index=False)


In [65]:
a = [1,2,3,4]
b = [5,6,7,8]
a.extend(b)
a

[1, 2, 3, 4, 5, 6, 7, 8]

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Function to get HTML of a Steam page
def get_steam_page(game_id):
    url = f"https://store.steampowered.com/app/{game_id}"
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        return None

# Function to parse Steam metadata
def parse_metadata(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Extract game title
    title = soup.find('div', class_='apphub_AppName').text.strip()
    
    # Extract developer
    developer = soup.find('div', class_='dev_row').find_all('a')[0].text.strip()
    
    # Extract genre
    genre = soup.find('div', class_='details_block').find_all('a')[1].text.strip()
    
    # Extract price
    try:
        price = soup.find('div', class_='game_purchase_price').text.strip()
    except:
        price = "Free"  # Some games are free
    
    # Extract release date
    release_date = soup.find('div', class_='date').text.strip()
    
    return {
        
        "Title": title,
        "Developer": developer,
        "Genre": genre,
        "Price": price,
        "Release Date": release_date
    }

# Function to store the data in a DataFrame
def store_data(game_data_list):
    df = pd.DataFrame(game_data_list)
    df.to_csv('steam_games_data.csv', index=False)

# Example game IDs to scrape (you can add more)
game_ids = sorted(['730', '578080', '252490', '367520', '504230', '250900', '413150', '268910', '1145360', 	'646570'])  # CS:GO, PUBG, Rust (as examples)

# List to hold all game metadata
game_data_list = []

# Loop through game IDs and scrape data
for game_id in game_ids:
    html_content = get_steam_page(game_id)
    if html_content:
        game_data = parse_metadata(html_content)
        game_data_list.append(game_data)


PermissionError: [Errno 13] Permission denied: 'steam_games_data.csv'

In [3]:

# Store the scraped data in a CSV file
store_data(game_data_list)

print("Scraping completed! Check steam_games_data.csv for results.")


Scraping completed! Check steam_games_data.csv for results.
